In [ ]:
# Parameters
mode = 'train'
num_senses = 7
num_POS = 2
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.01
decay_steps = 500
decay_rate = 0.96
clip_norm = 1
clipping = True

In [ ]:
##Model for One-million dataset
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
pos = tf.placeholder('int32', [batch_size], name="pos")
mask1 = tf.constant([0, 0, 0, 0, -1000, -1000, -1000]) # modify this according to the number of senses and POS_tags
mask2 = tf.constant([-1000, -1000, -1000, -1000, 0, 0, 0]) # modify this according to the number of senses and POS_tags
mask = tf.concat([mask1,mask2], 0)
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)



with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x) 
    
    
### First Bidirectional Lstm starts
with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h = tf.concat([fw_h1, bw_h1], 2)

## First bidirectional Lstm ends
    


## Second Bidirectional lstm starts

with tf.variable_scope("lstm3"):
    cell_fw3 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw3 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw3 = tf.contrib.rnn.DropoutWrapper(cell_fw3, input_keep_prob=input_keep_prob)
    d_cell_bw3 = tf.contrib.rnn.DropoutWrapper(cell_bw3, input_keep_prob=input_keep_prob)
    
    (fw_h3, bw_h3), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw3, d_cell_bw3, h, sequence_length=x_len, dtype='float', scope='lstm3')
    h3 = tf.concat([fw_h3, bw_h3], 2)
    
## Second Bidirectional lstm ends
h_new = tf.expand_dims(h,3)
## Max_pooling
pooled = tf.layers.max_pooling2d(inputs=h_new, pool_size=[max_sent_size, 1], strides=2)
pooled_flat = tf.reshape(pooled, [batch_size, 2*hidden_size])

## Softmax for POS_tags
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_POS], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_POS]), name="b")
    drop_h = tf.nn.dropout(pooled_flat, input_keep_prob)
    logits_h = tf.matmul(drop_h, W) + b
    pos_tag = tf.argmax(logits_h, 1)


def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], mean=0.0, stddev=0.1, seed=0), name="W_att")
    c = tf.expand_dims(attention(h3[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h3[i], x_mask[i], W_att), 0)], 0)
        
        
        
with tf.variable_scope("softmax_layer"):
    W1 = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], mean=0.0, stddev=0.1, seed=0), name="W1")
    b1 = tf.Variable(tf.zeros([num_senses]), name="b1")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W1) + b1
    rev_logits = tf.placeholder('float', [batch_size, num_senses], name="rev_logits")
    for i in range(batch_size):
        for j in range(num_POS):
            dummy = tf.zeros([1, num_senses], tf.float)
            rev_logits[i] = case({tf.equal(pos[i],j): mask[j] + logits[i], default=dummy, exclusive=True)
    sense = tf.argmax(rev_logits, 1)       
        

    
loss1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_h, labels=pos))
loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=rev_logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)

        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss1 + loss2 + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

    

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)  

In [ ]:
# Splitting
data_x = full_data[sense_word][0]
data_pos = full_data[sense_word][1]
data_y = full_data[sense_word][2]
x_train, x_test, y_train, y_test, pos_train, pos_test = train_test_split(data_x, data_y, data_pos, train_size=0.8, shuffle=True, stratify=data_y, random_state=0)

x_train, x_val, y_train, y_val, pos_train, pos_val = train_test_split(x_train, y_train, pos_train, train_size=0.9, shuffle=True, stratify=y_train, random_state=0)